In [1]:
def check_gpu():
    import tensorflow as tf
    print("tf.test.is_built_with_cuda()")
    print(tf.test.is_built_with_cuda())
    
    print()
    print("tf.config.list_physical_devices('GPU')")
    print(tf.config.list_physical_devices('GPU'))

    print()
    print("tf.config.experimental.list_physical_devices('GPU')")
    print(tf.config.experimental.list_physical_devices('GPU'))

#check_gpu()

In [2]:
import pandas as pd

chess_evals = pd.read_csv("chessData.csv").head(10000)
random_evals = pd.read_csv("random_evals.csv").head(10000)
tactic_evals = pd.read_csv("tactic_evals.csv").head(10000)
df = pd.concat([chess_evals, random_evals, tactic_evals], join="inner")
df = df.sample(frac=1, random_state=42)  # frac=1 shuffles all rows

def isEvaluationNumeric(myStr):
    try:
        myStr = myStr.lstrip('-')
        x = int(myStr)
        return True
    except:
        return False

# Convert the Evaluation column to integers and remove rows where conversion is not possible
df = df[df['Evaluation'].apply(lambda x: isEvaluationNumeric(x))]
df['Evaluation'] = df['Evaluation'].astype(int)

# Invert black evals
#df['Evaluation'] = df.apply(lambda row: -row['Evaluation'] if ' b ' in row['FEN'] else row['Evaluation'], axis=1)

def clamp(x):
    if x <= -1000:
        return -1000
    if x >= 1000:
        return 1000
    return x
# Clamp
df['Evaluation'] = df.apply(lambda row: clamp(row["Evaluation"]), axis=1)

def normalize(number, min_range, max_range):
    scaled_value = -1 + 2 * (number - min_range) / (max_range - min_range)
    return scaled_value
# Normalize to between -1 and 1
#df['Evaluation'] = df.apply(lambda row: normalize(row["Evaluation"], -10, 10), axis=1)

print(df.shape)
print("Avg eval:", round(df["Evaluation"].mean()))

(26744, 2)
Avg eval: -7


In [3]:
def generateData():
    from stockfish import Stockfish
    f = open("data.csv", "w")
    f.write("FEN,Evaluation")        
    stockfish = Stockfish(path="/users/ricar/desktop/stockfish/stockfish-windows-x86-64-avx2.exe",
                          depth=9, 
                          parameters={"Threads": 4})
    i = 0
    for fen in df["FEN"].values:
        eval = stockfish.get_evaluation()
        if eval["type"] != "cp":
            continue
        value = clamp(int(eval["value"]))
        f.write(fen + "," + str(value))
        print(str(i+1) + " evaluated")
        
generateData()

{'type': 'cp', 'value': 43}


In [ ]:
import numpy as np
import chess

def fen_to_12_8x8(myFen):
    whiteToMove = " w " in myFen
    if whiteToMove:
        board = chess.Board(myFen)
    else:
        splitted = myFen.split(" ")
        splitted[0] = splitted[0].swapcase() # Flip piece colors
        # Now we have to invert the rows
        rows = splitted[0].split("/")
        leftPartReversed = ""
        for i in range(len(rows)-1, -1, -1): # start stop step
            leftPartReversed += rows[i]
            if i != 0:
                leftPartReversed += "/"
        splitted[0] = leftPartReversed
        fen = " ".join(splitted)
        board = chess.Board(fen)
    
    pieces = [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]
    array = np.empty((12,8,8), dtype=int)
    i = 0 
    for color in [chess.WHITE, chess.BLACK]:
        for piece in pieces:
            bitboard = np.asarray(board.pieces(piece, color).tolist()).astype(int)
            bitboard = bitboard.reshape(8,8)
            array[i] = bitboard
            i += 1
            
    return array

# rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
#fen_to_12_8x8("7k/7p/6p1/4pp2/4P3/8/3Q1PPP/7K w - - 0 1")
#print()
#fen_to_12_8x8("7k/7p/6p1/4pp2/4P3/8/3Q1PPP/7K b - - 0 1")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X = np.array([fen_to_12_8x8(fen) for fen in df['FEN'].values])
y = df['Evaluation'].values
del df
y = y.reshape(y.shape[0],1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


In [ ]:
import pickle

def train():
    from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.regularizers import l2
    
    #model = Sequential()
    #model.add(Dense(units=32, activation='relu', input_dim=769))
    #model.add(Dense(units=1, activation="linear"))

    model = Sequential()
    model.add(Flatten(input_shape=(12, 8, 8)))  # Flatten the 3D input to a 1D array
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    model.summary()
        
    model.compile(loss="mae", optimizer=Adam(learning_rate=0.01)) # default lr 0.001
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

    model.save("model.h5")
    with open("history", 'wb') as hist:
        pickle.dump(history.history, hist) 
    
train()

In [ ]:
from tensorflow.keras.models import load_model
    
model = load_model("model.h5")
with open("history", "rb") as hist_file:
    history = pickle.load(hist_file)
model.summary()

In [ ]:
print("Evaluating on test set...")
loss = model.evaluate(X_test, y_test)

In [ ]:
def plot():
    import matplotlib.pyplot as plt
    global history
    if not isinstance(history, dict):
        history = history.history
    # Plot training and validation loss over epochs
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss (RMSE)')
    plt.legend()
    plt.show()
    
plot()

In [ ]:
def printWeightsForCpp(model):
    for layer in model.layers:
        print("------ LAYER", layer.name, "------")
        weights = layer.get_weights()
        if len(weights) > 0:
            rows, cols = weights[0].shape
            if cols > 1:
                print(f"double {layer.name}[{rows}][{cols}] = {{", end="")
                for row in weights[0]:
                    print("{", ",".join(str(val) for val in row), "},", end="")
            else:
                print(f"double {layer.name}[{rows}] = {{", end="")
                for row in weights[0]:
                    print(",".join(str(val) for val in row), ",", end="")

            print("};")
            print()
        print()
        
printWeightsForCpp(model)

In [ ]:
def printBiasesForCpp(model):
    for layer in model.layers:
        print("------ LAYER", layer.name, "------")
        biases = layer.get_weights()[1]
        print(f"double biases[{len(biases)}] = {{", end="")
        print(",".join(str(val) for val in biases), ",")
        print()
        
printBiasesForCpp(model)

In [ ]:
input = np.array([i % 4 for i in range(769)])
input = input.reshape(1, 769)

predictions = model.predict(input)
print("Prediction:", predictions[0][0])